Ternary trees is similar to Btrees in spirit but they have a little more data in the sense that they don't store values as such but they store characters. Whereas Btrees() can store anything, a ternary trees is intended to store strings. That's its purpose, so it's a little less general purpose.

A ternary search tree has nodes with the following attributes:
* a character, can be `None`;
* a Boolean flag that indicates whether the character represented
  by this node has been the last in a string that was inserted in the
  tree;
* the "less-than" child;
* the "equals" child and
* the "larger-than" child.

The data structure should support the following operations:
* string insert
* string search
* prefix string search
* return the number of strings stored in the data structure
* return all strings stored in the data structure

Also ensure that an instance of the data structure can be visualy represented, e.g., in aSCII format.

# Implementation

In this case, it is implemented in one/two python files, as a module which we need to write. We can either implement it as a module or do the implementation in the notebook

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The data structure has been implemented as a class.

In [26]:
from ternary_search_tree import TernarySearchTree

In [1]:
class TtreeNode:
    
    def __init__(self, char):
        self.root = None
        self.string = None
        #self._word = char  # to store in list of all words?
        self._char = char[0]  # value already stored in node x, store as attribute of this node
        self._lt, self._gt, self._eq = None, None, None
        self.flag_wordend = False  # mark the end of a word

    def _all_strings(self, pf=''):
        #print(f'printing {self} with prefix {pf}')
        final_wordlist = []
        
        word = pf + self._char

        # missing: if prefix empty, add it as word

        if self.flag_wordend:
            final_wordlist.append(word)

        if self._lt is not None:
            #print(self._lt)
            final_wordlist.extend(self._lt._all_strings(pf))
        if self._gt is not None:
            #print(self._gt)
            final_wordlist.extend(self._gt._all_strings(pf))
        if self._eq is not None:
            #print(self._eq)
            words = self._eq._all_strings(word)
            final_wordlist.extend(words)

        return final_wordlist
    
    def __len__(self):
        # should count keys, not nodes
        if self.flag_wordend:
            length = 1
        else:
            length = 0
        # add edge case if string is empty
        # if self._char == "":
        #     length += 1

        if self._eq is not None:
                length += len(self._eq)
        if self._lt is not None:
                length += len(self._lt)
        if self._gt is not None:
                length += len(self._gt)
        
        return length

    def _to_string(self, indent=' '):
        terminates = f'Terminates: {self.flag_wordend}'
        print_info = f'char: {repr(self)}, '
        #repr_str = indent + repr(self)
        repr_str = indent + print_info + indent + terminates
        if self._eq is not None:
            repr_str += '\n' + '_eq:' + self._eq._to_string(indent + '  ')
        if self._lt is not None:
            repr_str += '\n' + '_lt:' + self._lt._to_string(indent + '  ')
        if self._gt is not None:
            repr_str += '\n' + '_gt:' + self._gt._to_string(indent + '  ')
        return repr_str
    
    def __repr__(self):
        # remove star later, for now just for debugging
        return f"{self._char}{'*' if self.flag_wordend else ''}"
    
    def _insert(self, string):
        #print(f'inserting {string} at node {self._char}')
        # string is the new key to insert
        # self.string is the value already stored in this node,  holds the current key in this node
        self.string = string  # only the first time the function is called -> add flag as function argument
        
        if len(string) > 0:
            char = string[0]  # first character of new word
            rest = string[1::]  # if rest as list: char, *rest = string
        else:
            char = string
            rest = string

        # if string contains more than one character
        if len(string) > 1:
            # character matches
            if char == self._char:

            # insert in middle child
                if self._eq is None:
                    self._eq = TtreeNode(rest[0])
                self._eq._insert(rest)

            # if earlier in the alphabet:
            elif char < self._char:
                if self._lt is None:
                    self._lt = TtreeNode(char)
                self._lt._insert(string)
            # if later in the alphabet
            elif char > self._char:
                if self._gt is None:
                    self._gt = TtreeNode(char)
                self._gt._insert(string)
        #elif string == "":
        #    if self._eq is None:
        #        self._eq = TtreeNode(string)
        #    self._eq._insert(string)
        else:
            self.flag_wordend = True
        
        return string
    
    def _psearch(self, string):
        self.string = string
        #print(self._char)

        if len(string) > 1:
            char = string[0]  # first character of search word
            rest = string[1::]
        else:  # necessary?
            char = string
            rest = string
        
        print(f'searching for char {char} at node {self._char}')

        # if character is found:
        if char == self._char:
            if len(string) == 1:
                return self
            elif len(string) > 1:
                if self._eq is not None:
                    return self._eq._psearch(rest)

        elif char < self._char:
            if self._lt is not None:
                return self._lt._psearch(string)

        elif char > self._char:
            if self._gt is not None:
                return self._gt._psearch(string)

In [2]:
class TernarySearchTree:
    
    def __init__(self):
        self._root = None

    def all_strings(self):
        if self._root is None:
            return []
        else:
            return self._root._all_strings()
        
    def __len__(self):
        print(f'printing length of {self._root}')
        if self._root is None:
            return 0
        else:
            return len(self._root)
    
    def __repr__(self):
        if self._root is None:
            return 'empty tree'
        else:
            #print("print string here")
            return self._root._to_string('')
    
    def insert(self, string):
        if self._root is None:
            #print(f'inserting {string} at node {self._root}')
            self._root = TtreeNode(string)
        self._root._insert(string)


    def search(self, prefix):
        if self._root is None:
            return False
        node = self._root._psearch(prefix)
        #print(node)
        if node is None:
            return []
        elif node._eq:
            # keep recursing into middle children as long as there is one
            # return all words that contain the prefix
            return node._eq._all_strings(prefix)
        elif node.flag_wordend:
            return [prefix]
        else:
            return node


# Example usage

Create a new empty ternery search tree.

In [3]:
tst = TernarySearchTree()

In [4]:
print(tst)

empty tree


Insert the string `'abc'` into the tree.

In [5]:
tst.insert('abc')

Display the tree.

In [6]:
print(tst)

char: a, Terminates: False
_eq:  char: b,   Terminates: False
_eq:    char: c*,     Terminates: True


Insert another string `'aqt'`.

In [7]:
tst.insert('aqt')

In [8]:
print(tst)

char: a, Terminates: False
_eq:  char: b,   Terminates: False
_eq:    char: c*,     Terminates: True
_gt:    char: q,     Terminates: False
_eq:      char: t*,       Terminates: True


The tree should now contain two strings.

In [9]:
len(tst)

printing length of a


2

In [10]:
tst.all_strings()

['aqt', 'abc']

Search for the string `'ab'`, it should be found since it is a prefix of `'abc'`.

In [11]:
tst.search('ab')

searching for char a at node a
searching for char b at node b


['abc']

The string `'ac'` should not be found.

In [12]:
tst.search('ac')

searching for char a at node a
searching for char c at node b
searching for char c at node q


[]

The tree can also contain the empty string.

In [17]:
tst.insert('')

In [18]:
len(tst)

printing length of a*


3

In [19]:
print(tst)

char: a*, Terminates: True
_eq:  char: b,   Terminates: False
_eq:    char: c*,     Terminates: True
_gt:    char: q,     Terminates: False
_eq:      char: t*,       Terminates: True


In [20]:
tst.all_strings()

['a', 'aqt', 'abc']

# Testing

The worst case for the quicksort algorithm is the sorted list. Maybe that's something we should remember if we look at this dataset (a hint from GJB).

In this project we are supposed to implement similarly to the Btree. Also, do proper testing that your algorithm actually works as expected. Also, pay attention to Corner cases: what happens if I have an empty ternary tree, does the right thing happen (i.e. do I get correct values for the length, do I get correct values for the strings that are sorted in there, etc...). Make sure to have tests in place to test for these cases. The third thing is the performance test: how does it scale with increasing number of words stored in it, how long does it take to build the ternary search tree, also how long does it take to find stuff (in worst case too). Basically, similar tests as we did for the binary tree or the sorting algorithm.

The whole thing is supposed to be implemented using version control. Hence, everything lives in a Github repository. Also there should be documentation in your implementation, just as the documentation seen in the sorting thing.   

Discussion should also be there, i.e., you see something -> comment on what you see. Try to explain what you see, whether you expect it. 

The file `data/search_trees/insert_words.txt` contains words that we can insert into a tree.

In [21]:
tst = TernarySearchTree()
with open('data/search_trees/insert_words.txt') as file:
    words = [
        line.strip() for line in file
    ]
for word in words:
    tst.insert(word)
unique_words = set(words)

Verify the length of the data stucture.

In [22]:
assert len(tst) == len(unique_words), \
       f'{len(tst)} in tree, expected {len(unique_words)}'

printing length of c*
printing length of c*


AssertionError: 19 in tree, expected 20

Verify that all words that were inserted can be found.

In [23]:
for word in unique_words:
    assert tst.search(word), f'{word} not found'

searching for char f at node c
searching for char f at node d
searching for char f at node f
searching for char a at node u
searching for char a at node o
searching for char a at node a
searching for char r at node r
searching for char a at node c
searching for char a at node b


AssertionError: a not found

Verify that all prefixes can be found.

In [24]:
for word in unique_words:
    for i in range(len(word) - 1, 0, -1):
        prefix = word[:i]
        assert tst.search(prefix), f'{prefix} not found'

searching for char f at node c
searching for char f at node d
searching for char f at node f
searching for char a at node u
searching for char a at node o
searching for char a at node a
searching for char f at node c
searching for char f at node d
searching for char f at node f
searching for char c at node c
searching for char o at node o
searching for char m at node m
searching for char b at node b
searching for char i at node i
searching for char n at node n
searching for char a at node e
searching for char a at node a
searching for char t at node t
searching for char i at node i
searching for char o at node o
searching for char c at node c
searching for char o at node o
searching for char m at node m
searching for char b at node b
searching for char i at node i
searching for char n at node n
searching for char a at node e
searching for char a at node a
searching for char t at node t
searching for char i at node i
searching for char c at node c
searching for char o at node o
searchin

Check that when searching for a exact match, only the inserted words are found, and no prefixes.

In [44]:
for word in unique_words:
    for i in range(len(word), 0, -1):
        prefix = word[:i]
        if prefix not in unique_words:
            assert not tst.search(prefix, exact=True), \
                   f'{prefix} found'

Check that the empty string is in the tree (since it is a prefix of any string).

In [45]:
assert tst.search(''), 'empty string not found'

Check that the empty string is not in the tree for an exact search.

In [46]:
assert not tst.search('', exact=True), 'empty string found'

Check that words in the file `data/search_trees/not_insert_words.txt` can not be found in the tree.

In [47]:
with open('data/search_trees/not_insert_words.txt') as file:
    for line in file:
        word = line.strip()
        assert not tst.search(word), f'{word} should not be found'

Check that all strings are returned.

In [48]:
all_strings = tst.all_strings()
assert len(all_strings) == len(unique_words), \
       f'{len(all_strings)} words, expected {len(unique_words)}'
assert sorted(all_strings) == sorted(unique_words), 'words do not match'

If not output was generated, all tests have passed.